# Markov chains

A first-order Markov chain is a conditional probability distribution of the form
$$
    P(X_k = x_k | X_{k-1} = x_{k-1}),
$$
where $x_j \in X$, $X$ is the support of $X_1,X_2,\ldots$.
In this context, we refer to $X$ as the set of *tokens* that may be observed.

An $n$-th order Markov chain models
$$
    P(X_k = x_k | x_{k-1}, x_{k-2}, \ldots, x_{k-n + 1}).
$$

We can model this with a first-order Markov chain with $|X|^n$ states.
Some observations:

1. As the token set $X$ increases in size (cardinality), the number of states
grows by polynomially, $O(|X|^n)$.

2. As the order $n$ increases, the number of states grows exponentially, but the
   non-zero state transition probabilities stays *constant*, $O(|X|)$ and therefore transition matrix becomes increasingly sparse.

Sparse matrices are pretty efficient to represent, but the number of states
can be a problem.

We consider an AR language model over bytes (256 tokens at max, although we
only populate whatever is in the training data),

$$
    P(X_k = x_k | x_{k-1}, x_{k-2}, \ldots, x_{k-n + 1})
$$
where each $x_i$ is a byte token, typically printable ASCII characters.

We can thus think of this as an $n$-th order Markov chain with $O(256^n)$ states,
also known as an $n$-gram model.

> Technically, we actually store states with
> $n-1$ tokens down to $0$ tokens, also, so the number of states is actually
> $O(256^{n+1})$.

In our model, we never enter a state, here called a *context*, that has not been
encountered in the training data. The context is simply the last $n$ tokens observed,
or less if we have not observed $n$ tokens yet. Since we may not use most states,
and the transition matrix is sparse, it may make sense to represent the transition
matrix as a dictionary of dictionaries, where the outer dictionary is indexed by
the context and the inner dictionary is indexed by the token and contains the
number of times that token was observed in that context in the training data.
We can then convert this to a probability distribution by normalizing the counts
in each context.

We can then generate text by starting with a any context and then sampling from
the probability distribution for that context to get the next token. We do something
pretty silly in our construction -- we keep the entire history of tokens around
to generate the next token, because we do not hard code the order of the Markov
chain and allow it to vary for each observation or training data set.

This kind of language model, the $n$-gram model, is a simple model. Compared
to more sophisticated models, like transformer-based models, it performs quite
poorly. Here are a few reasons why:

1. The $n$-gram model is not able to capture long-range dependencies in the data
as well, given that the number of states grows exponentially with the order
of the model, which makes it difficult to use for large $n$.

2. The $n$-gram model does not generalize well to unseen data. Since language
is a high-dimensional space, *most* contexts are unseen in the training data.
We represent an additional inductive bias by assuming that if a context was
not seen previously, the next-best context is one in which the oldest token
is dropped. This is a pretty strong assumption, and is not generally true,
but it is a simple way to handle unseen contexts and it's often reasonable.

Due to points (1) and (2) above, the $n$-gram model does not in practice capture
the semantics of a naturla language very well. Presumably, if we had
a sufficiently large training data set with a sufficiently large order $n$ for
context, we could capture the semantics of the language. However, natural
language is far too high-dimensional to represent by simply storing it; a good
predictive model is one which **compresses** the representation
of the data such that it can be efficiently reconstructed. This is the human
brain operates -- we don't remember every single thing we've ever seen, but
we remember a compressed representation of it such that we can fit some salient
mental model of the world in our tiny little heads. This touches upon the
hypothetical concept of *compression* as a proxy for *understanding* in the
context of machine learning. To compress something, you must be able to
extract the salient features and use those features to reconstruct the original
data or predict future data.

In neural linguistic models, we use a large neural network to learn a compressed
representation of the data, and then use that representation to predict future
data. These compressed representations naturally do things like map
semantically similar words and phrases to similar representations, and thus
if we prompt the model with a prompt that is semantically similar to the training
data, it will be able to more accurately predict the next token. This is because
the model has learned a compressed representation of the data that captures
the semantics of the language.

However, that said, here we explore this simple $n$-gram model over bytes, which
may help us understand some aspects of how large language models work. Since
natural language is too complex to model with a simple $n$-gram model, we train
it on synthetic data, or algorithmic data, to see how well different orders of
the model can capture the structure of the data and use it to generate new
data. Indeed, we can use it by prompting it in the same way we prompt
LLMs, and see how well it can predict the correct output, because in our data
geenerating process (DGP), the data at some points is *deterministic*, such as
when given the prompt `sort[1, 3, 2]`, the correct output is `[1, 2, 3]` without
exception in the DGP. As we shorten the context, or the order, we see that the
model becomes less and less able to predict the correct output, and the
generated data becomes more and more random, as a distribution over the now
shorter context. We can see that the context that was removed was salient but
now *latent* in the data, and the model is unable to capture the full structure
of the data.

Even large language models have his problem. Even if we *assume* they had
a sufficiently large context and model capacity, they still have two problems:

(1) They do not necessarily generalize out-of-distribution as well as they
could. This is because the data they are trained on is not necessarily
representative of the data they will be used on

(2) Because most states are out-of-distribution in these high-dimensioal
state spaces, the model must generalize to unseen states. However, the model
may not capture the optimal set of inductive biases for the task at hand. There
is a sayinug in the ML community: there are no free lunches. This means that
there is no model that is optimal for all tasks, and that all models have
inductive biases that make them better at some tasks and worse at others. The
trick is to find the right model for the right task.

In [9]:
import algorithmic_data
import importlib
import mc2
import math

importlib.reload(mc2)
importlib.reload(algorithmic_data)
data = algorithmic_data.generate([
   max, min, sorted, sum, math.prod], 10)


algorithmic_data.generate_tree([max, min, sum, sorted, id, math.prod],
                               .01, 5, range(10))

{'result': 6, 'data': '2'}

In [10]:
data[110:120]

[]

In [11]:
importlib.reload(algorithmic_data)
data2 = algorithmic_data.generate_comp([sum,min,max,sorted],2000000)

#print(algorithmic_data.generate_tree([sum, min, max, sorted], 3, 3, 9, '.'))

In [12]:
data2[:4]

['sum[sorted[7,4],min[5,4,7]]=15.',
 'sum[sorted[2,2,7],sum[5,1,1]]=18.',
 'sum[sorted[7,0],max[0,4,4]]=11.',
 'sorted[sorted[3,0],min[7,9]]=[0,3,7].']

In [13]:
mc = mc2.MarkovChain()
mc.train(data, order = 20)

In [90]:
# let's generate a table showing sum, sorted, prod, max, and min
# this table will be based on the final completion
# after the `=`, e.g
# 



mc.generate('sorted[1,9,3,4]=')

mc.generate('min[2,max[1,1,1,1]]=')

'12.'